In [1]:
from gseapy.parser import Biomart
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import os
import glob

In [2]:
# use biomart
# read the list of deseq genes
# use prerank
# plot gsea

In [3]:
# conditions of interest:
#     LPS0_shCTRL_vs_shMOF
#     LPS0_shCTRL_vs_shPRDX1
#     LPS3_shCTRL_vs_shMOF
#     LPS3_shCTRL_vs_shPRDX1
#     LPS12_shCTRL_vs_shMOF
#     LPS12_shCTRL_vs_shPRDX1

In [4]:
in_path = os.path.join("/data/processing1/leily/deseq_pairwise/fdr0.05")
out_path = os.path.join("/data/processing1/leily/deseq_pairwise/fdr0.05/gsea")

In [5]:
# From GSEApy developer:
# prerank is used for comparing two group of samples (e.g. control and treatment),
# where the gene ranking are defined by your custom rank method (like t-statistic, signal-to-noise, et.al).

In [6]:
# convert IDs using biomart
from gseapy.parser import Biomart
bm = Biomart()
marts = bm.get_marts()
datasets = bm.get_datasets(mart='ENSEMBL_MART_ENSEMBL')

In [7]:
genes_names = pd.read_csv(os.path.join(out_path, "gene_id_ensembl.tsv"), sep = "\t")
queries = genes_names["gene_id"].values.tolist() # need to be a python list
print(len(queries))
results = bm.query(dataset='mmusculus_gene_ensembl',
                   attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id'],
                   filters={'ensembl_gene_id': queries})

47937


In [8]:
for time in ["lps0", "lps3","lps12"]:
    for file in glob.glob(os.path.join(in_path, "ddr_"+time+"_shctrl*.filtered.tsv")):
        name = os.path.basename(file)
        name = name.split(".filtered")[0].split("ddr_")[-1]
        print(name)
        df = pd.read_csv(file, sep = "\t", usecols = ["gene_id", "log2FoldChange"])
        df.sort_values(by=["log2FoldChange"], inplace = True, ascending = False)
        df['ensembl_gene_id'] = df['gene_id'].str.split('.', 1).str[0]
        merged_df = df.merge(results, on = "ensembl_gene_id", how = "inner")
        print(len(merged_df))
        df1 = pd.DataFrame()
        df1[0] = merged_df["external_gene_name"].str.upper()
        df1[1] = merged_df["log2FoldChange"]
        df1.dropna(inplace = True)
        df1.sort_values(by=[1], ascending = False, inplace = True)
        pre_res = gp.prerank(rnk=df1, gene_sets='KEGG_2019_Mouse',
                             processes=4,
                             seed=149,
                             graph_num = 25,
                             permutation_num=100,
                             outdir=os.path.join(out_path,'test_prerank_report_kegg_2019_'+name))

lps0_shctrl_lps0_shprdx1


2021-09-15 16:01:20,939 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


44210
lps0_shctrl_lps0_shmof
44210


2021-09-15 16:02:45,643 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


lps3_shctrl_lps3_shmof


...

44210
lps12_shctrl_lps12_shprdx1


2021-09-15 16:06:57,063 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


44210


2021-09-15 16:08:21,705 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


lps12_shctrl_lps12_shmof
44210


In [9]:
# look for common pathways within a time point

In [10]:
for time in ["lps0", "lps3", "lps12"]:
    merged_df = pd.DataFrame()
    for index, file in enumerate(glob.glob(os.path.join(out_path, "test_prerank_*"+time+"*", "*csv"))):
        print(file)
        df = pd.read_csv(file, sep =",", usecols = ["Term", "nes", "pval", "fdr"])
        print(len(df))
        df = df.loc[df["fdr"] <= 0.25] # recommneded by developer!
        print(len(df))
        if index == 0:
            merged_df = df
        else:
            merged_df = merged_df.merge(df, how = "inner", on = "Term")
    print(merged_df)

/data/processing1/leily/deseq_pairwise/fdr0.05/gsea/test_prerank_report_kegg_2019_lps0_shctrl_lps0_shmof/gseapy.prerank.gene_sets.report.csv
288
10
/data/processing1/leily/deseq_pairwise/fdr0.05/gsea/test_prerank_report_kegg_2019_lps0_shctrl_lps0_shprdx1/gseapy.prerank.gene_sets.report.csv
288
1
Empty DataFrame
Columns: [Term, nes_x, pval_x, fdr_x, nes_y, pval_y, fdr_y]
Index: []
/data/processing1/leily/deseq_pairwise/fdr0.05/gsea/test_prerank_report_kegg_2019_lps3_shctrl_lps3_shmof/gseapy.prerank.gene_sets.report.csv
288
24
/data/processing1/leily/deseq_pairwise/fdr0.05/gsea/test_prerank_report_kegg_2019_lps3_shctrl_lps3_shprdx1/gseapy.prerank.gene_sets.report.csv
288
4
                                   Term     nes_x    pval_x     fdr_x  \
0              ECM-receptor interaction  1.514909  0.000000  0.182546   
1  Maturity onset diabetes of the young  1.496396  0.029412  0.199846   

      nes_y    pval_y     fdr_y  
0  1.611732  0.038462  0.184684  
1  1.697866  0.000000  0.123122 

In [11]:
# filter those with FC>0.5 (FC<0.05)+padi<0.05
# gp.enrichr

In [12]:
up_dfs = dict()
down_dfs = dict()
for time in ["lps0", "lps3", "lps12"]:
    for cond in ["shmof", "shprdx1"]:
        for file in glob.glob(os.path.join(in_path, "ddr_"+time+"_shctrl_"+time+"_"+cond+".filtered.tsv")):
            df = pd.read_csv(file, sep = "\t")
            df = df.loc[df["padj"] < 0.05]
            df['ensembl_gene_id'] = df['gene_id'].str.split('.', 1).str[0]
            merged_df = df.merge(results, on = "ensembl_gene_id", how = "inner")
            df_up = merged_df.loc[merged_df["log2FoldChange"] > 0.05]
            df_down = merged_df.loc[merged_df["log2FoldChange"] < 0.05]
            if time not in up_dfs.keys():
                up_dfs[time] = dict()
                down_dfs[time] = dict()
            
            up_dfs[time][cond] = df_up
            down_dfs[time][cond] = df_down
print(down_dfs["lps0"]["shprdx1"])

                    gene_id     baseMean  log2FoldChange     lfcSE       stat  \
50    ENSMUSG00000025791.18  1116.730995       -1.158904  0.060829 -19.051759   
53     ENSMUSG00000045362.8   605.639786       -1.719134  0.090601 -18.974808   
68    ENSMUSG00000022351.14  1263.547857       -1.312750  0.072620 -18.076869   
69    ENSMUSG00000031511.14  4063.773293       -0.768367  0.042593 -18.039582   
72    ENSMUSG00000042079.13  4489.832676       -0.703400  0.039579 -17.772149   
...                     ...          ...             ...       ...        ...   
6302   ENSMUSG00000023932.8  1882.724461       -0.108514  0.046587  -2.329262   
6303   ENSMUSG00000097357.1    25.878429       -0.820373  0.352319  -2.328496   
6304   ENSMUSG00000041297.7  2130.140977       -0.134378  0.057713  -2.328390   
6306  ENSMUSG00000018377.10  2537.483579       -0.129075  0.055463  -2.327239   
6308   ENSMUSG00000028330.8  1201.941981       -0.144548  0.062132  -2.326474   

            pvalue         

In [20]:
for time in ["lps0", "lps3", "lps12"]:
    for cond, df in up_dfs[time].items():
        enr = gp.enrichr(gene_list=df["external_gene_name"].astype(str),
                          description=time+"_"+cond+"_up",
                          gene_sets='KEGG_2019_Mouse',
                          background= 30000, # the number of genes, e.g 20000
                          outdir=os.path.join(out_path,'enricher_report_kegg_2019_'+time+"_"+cond+"_up"),
                          cutoff=0.05, # only used for testing.
                          organism='Mouse',
                          verbose=False)
#         dotplot(enr.res2d, title='KEGG_2019_Mouse',cmap='viridis_r')

--- Logging error ---
Traceback (most recent call last):
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/data/proce

--- Logging error ---
Traceback (most recent call last):
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/data/proce

--- Logging error ---
Traceback (most recent call last):
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/data/proce

--- Logging error ---
Traceback (most recent call last):
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/data/proce

In [13]:
# Could you help me to sort out up-regulated genes in TNF pathway in LPS3_shCTRL_vs_shMOF and 
# LPS3_shCTRL_vs_shPRDX1 and collect their log2FC from DESeq2 in a file?
gene_name_df = pd.read_csv("/data/processing1/leily/deseq_pairwise/fdr0.05/gsea/kegg_name.tsv", sep ="\t")

In [14]:
# tnf path ledge_genes in mof (108 genes)
tnf_mof = ["CXCL10", "CXCL3", "PTGS2", "CXCL1", "IL6", "CSF2", "CXCL2", "TRAF1","MMP3", "CSF1", "VCAM1",
           "CCL20", "SELE", "TNFAIP3", "IL15", "CX3CL1", "LIF", "CCL2", "IFNB1", "IL1B", "MAP2K3", "MAPK13",
           "JAG1", "MAP2K6", "NOD2", "ICAM1", "RPS6KA5", "DAB2IP", "CREB3L2", "SOCS3", "MAPK12", "NFKB1",
           "TNFRSF1B", "FAS", "MAP2K4", "FOS", "NFKBIA", "CASP3", "IRF1", "JUNB", "TNF", "MAP2K1", "PIK3R3",
           "CREB5", "CREB3L4", "CFLAR", "BAG4", "MAP3K5", "BCL3", "IKBKB", "TAB3", "TRAF5", "RELA", "BIRC3",
           "TRAF2", "GM5431", "BIRC2", "PGAM5", "AKT3", "ITCH", "ATF4", "IKBKG", "PIK3R2", "ATF2", "CHUK",
           "CASP7", "PIK3R1", "TRAF3", "AKT1", "TRADD", "MAP3K7", "MAPK3", "MAP2K7", "CREB1", "CREB3L1",
           "MAPK8", "AKT2", "IFI47", "RIPK1", "CEBPB", "DNM1L", "MLKL", "PIK3CB", "MAPK1", "FADD", "LTA",
           "TAB1", "ATF6B", "MAP3K8", "CCL5", "PIK3CA", "RIPK3", "EDN1", "TAB2", "CASP8", "TNFRSF1A",
           "JUN", "CREB3", "MAPK11", "MAPK9", "MAP3K14", "MMP9", "IL18R1", "MAPK14", "CREB3L3", "PIK3CD",
           "CCL12", "MMP14"]
tnf_mof_ensemble = gene_name_df.loc[gene_name_df["external_gene_name"].str.upper().isin(tnf_mof)]
tnf_mof_ensemble =tnf_mof_ensemble[["ensembl_gene_id", "ensembl_gene_id_version", "external_gene_name"]]

In [15]:
# tnf path ledge_genes in prdx1 (108 genes)
tnf_prdx1 = ["CCL20", "CXCL3", "CASP7", "MMP9", "CXCL1", "IL1B", "CSF2", "CXCL2", "PTGS2", "IL6",
             "VCAM1", "RPS6KA5", "JAG1", "NFKBIA", "MAP3K5", "TNF", "TNFAIP3", "CREB3L2", "ICAM1",
             "MAPK11", "CXCL10", "NOD2", "IKBKB", "BIRC3", "IRF1", "TRAF1", "TAB3", "CREB3L1", "FAS",
             "BCL3", "RELA", "MAPK12", "MAP2K7", "PGAM5", "MAP3K7", "MMP14", "JUNB", "MAPK14", "CHUK",
             "JUN", "NFKB1", "AKT2", "AKT1", "CCL2", "IL15", "CFLAR", "IKBKG", "RIPK3", "PIK3CA", "TRADD",
             "BIRC2", "MAP2K6", "CEBPB", "MLKL", "MAPK13", "DNM1L", "ATF2", "IFNB1", "TRAF5", "ITCH",
             "CREB3L4", "ATF6B", "SOCS3", "DAB2IP", "CREB1", "MAPK1", "CCL5", "MAP2K4", "MAPK8", "PIK3R1",
             "IFI47", "MAP3K14", "MAP3K8", "MAP2K3", "TAB2", "LIF", "RIPK1", "BAG4", "ATF4", "MAPK9", "PIK3R2",
             "CSF1", "MAPK3", "CREB5", "AKT3", "PIK3R3", "TRAF3", "FOS", "TNFRSF1B", "CREB3", "FADD", "PIK3CB",
             "CASP3", "TNFRSF1A", "TRAF2", "CX3CL1", "TAB1", "MAP2K1", "CASP8", "GM5431", "MMP3", "CREB3L3",
             "SELE", "IL18R1", "EDN1", "PIK3CD", "CCL12", "LTA"]
tnf_prdx1_ensemble = gene_name_df.loc[gene_name_df["external_gene_name"].str.upper().isin(tnf_prdx1)]
tnf_prdx1_ensemble =tnf_prdx1_ensemble[["ensembl_gene_id", "ensembl_gene_id_version", "external_gene_name"]]

In [22]:
deseq_mof = os.path.join(in_path, "ddr_lps3_shctrl_lps3_shmof.filtered.tsv")
deseq_mof_df = pd.read_csv(deseq_mof, sep = "\t")
deseq_mof_df['ensembl_gene_id'] = deseq_mof_df['gene_id'].str.split('.', 1).str[0]
deseq_mof_df = deseq_mof_df.merge(tnf_mof_ensemble, on = "ensembl_gene_id", how = "inner")
deseq_mof_df.to_csv(os.path.join(out_path, "tnf_genes_mof_lp3.tsv"), sep = "\t", index = None)
deseq_prdx1 = os.path.join(in_path, "ddr_lps3_shctrl_lps3_shprdx1.filtered.tsv")
deseq_prdx1_df = pd.read_csv(deseq_prdx1, sep = "\t")
deseq_prdx1_df['ensembl_gene_id'] = deseq_prdx1_df['gene_id'].str.split('.', 1).str[0]
deseq_prdx1_df = deseq_prdx1_df.merge(tnf_prdx1_ensemble, on = "ensembl_gene_id", how = "inner")
deseq_prdx1_df.to_csv(os.path.join(out_path, "tnf_genes_prdx1_lp3.tsv"), sep = "\t", index = None)
